In [1]:
import os
import pandas as pd
import re

### Read the input files for the questions

In [2]:
trainData = pd.read_csv("archive/train.csv")
trainData.columns = trainData.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
print (trainData.shape)
validData = pd.read_csv("archive/valid.csv")
validData.columns = validData.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
print (validData.shape)
questions = trainData.append(validData)
print (questions.shape)

(45000, 6)
(15000, 6)
(60000, 6)


In [3]:
questions.head()

,id,title,body,tags,creationdate,y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ


### Clean up the data in the question body

In [4]:
questions['body'] = questions['body'].map(lambda x: re.sub('\\n',' ',str(x))) # Replace the new line characters
questions['body'] = questions['body'].map(lambda x: re.sub(r'\W',' ',str(x))) # Replace any non-word character i.e. anything not in 0-9, a-z, A-Z
questions['body'] = questions['body'].map(lambda x: re.sub(r'\s+[a-zA-Z]\s+',' ',str(x))) # Replace any whitespace other than a space, tab or newline
questions['body'] = questions['body'].map(lambda x: re.sub(r'\^[a-zA-Z]\s+',' ',str(x))) 
questions['body'] = questions['body'].map(lambda x: re.sub(r'\s+',' ',str(x)))
questions['body'] = questions['body'].str.lower()

questions['body'] = questions['body'].map(lambda x: re.sub(r"\’", "\'", str(x)))
questions['body'] = questions['body'].map(lambda x: re.sub(r"\n", "", str(x)))
questions['body'] = questions['body'].map(lambda x: re.sub(r"\r", "", str(x)))
questions['body'] = questions['body'].map(lambda x: re.sub(r"\'", "", str(x)))
questions['body'] = questions['body'].map(lambda x: re.sub(r"\"", "", str(x)))
questions['body'] = questions['body'].map(lambda x: re.sub(r'[?|!|\'|"|#]',r'', str(x)))
questions['body'] = questions['body'].map(lambda x: re.sub(r'[.|,|)|(|\|/]',r' ', str(x)))

In [5]:
questions.head()

,id,title,body,tags,creationdate,y
0,34552656,Java: Repeat Task Every Random Seconds,i already familiar with repeating tasks every...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,i like to understand why java 8 optionals wer...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,i am attempting to overlay title over an imag...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,the question is very simple but just could no...,<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,i using custom floatingactionmenu need to imp...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ


### Convert the tags to a list

In [6]:
tagsToList = lambda x: list(x.replace("<","").replace(">"," ").strip().split(" "))
questions['tags'] = questions['tags'].apply(tagsToList)

In [7]:
questions.head()

,id,title,body,tags,creationdate,y
0,34552656,Java: Repeat Task Every Random Seconds,i already familiar with repeating tasks every...,"[java, repeat]",2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,i like to understand why java 8 optionals wer...,"[java, optional]",2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,i am attempting to overlay title over an imag...,"[javascript, image, overlay, react-native, opa...",2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,the question is very simple but just could no...,"[swift, operators, whitespace, ternary-operato...",2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,i using custom floatingactionmenu need to imp...,"[android, material-design, floating-action-but...",2016-01-01 05:21:48,HQ


### Extract the primary tag from the question

In [24]:
getTag1 = lambda x: str(x).replace('[','').replace(']','').replace("'","").split(",")[0] if len(str(x).split(",") >= 1) else ""
getTag2 = lambda x: str(x).replace('[','').replace(']','').replace("'","").split(",")[1] if len(str(x).split(",") >= 2) else ""
getTag3 = lambda x: str(x).replace('[','').replace(']','').replace("'","").split(",")[2] if len(str(x).split(",") >= 3) else ""
getTag4 = lambda x: str(x).replace('[','').replace(']','').replace("'","").split(",")[3] if len(str(x).split(",") >= 4) else ""
getTag5 = lambda x: str(x).replace('[','').replace(']','').replace("'","").split(",")[4] if len(str(x).split(",") >= 5) else ""
questions['Tag1'] = questions['tags'].apply(getTag1)
questions['Tag1'] = questions['tags'].apply(getTag1)
questions['Tag1'] = questions['tags'].apply(getTag1)
questions['Tag1'] = questions['tags'].apply(getTag1)
questions['Tag1'] = questions['tags'].apply(getTag1)


In [26]:
questions.head()

,id,title,body,tags,creationdate,y,primaryTag
0,34552656,Java: Repeat Task Every Random Seconds,i already familiar with repeating tasks every...,"[java, repeat]",2016-01-01 00:21:59,LQ_CLOSE,java
1,34553034,Why are Java Optionals immutable?,i like to understand why java 8 optionals wer...,"[java, optional]",2016-01-01 02:03:20,HQ,java
2,34553174,Text Overlay Image with Darkened Opacity React...,i am attempting to overlay title over an imag...,"[javascript, image, overlay, react-native, opa...",2016-01-01 02:48:24,HQ,javascript
3,34553318,Why ternary operator in swift is so picky?,the question is very simple but just could no...,"[swift, operators, whitespace, ternary-operato...",2016-01-01 03:30:17,HQ,swift
4,34553755,hide/show fab with scale animation,i using custom floatingactionmenu need to imp...,"[android, material-design, floating-action-but...",2016-01-01 05:21:48,HQ,android


### Write to new file

In [27]:
questions.to_csv("Cleaned_Questions.csv")